In [2]:
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm
import json
import pandas as pd
import time
import dask.dataframe as dd
from math import sqrt
from pqdm.processes import pqdm
from joblib import Parallel, delayed
from tqdm import tqdm

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="./chromedriver.exe")

baseUrl = "https://hethongphapluat.com/hoi-dap-phap-luat_page-{}.html"
# result = pqdm(range(0, 100), crawl_parallel, n_jobs=5)


D:\tailieu\DOAN\crawlingdata\venv\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
D:\tailieu\DOAN\crawlingdata\venv\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: use options instead of chrome_options


In [4]:
df = pd.read_csv("./raw_data/raw_HeThongPhapLuat_ANS.csv", index_col=0)
df_answer = pd.read_csv("./raw_data/raw_HeThongPhapLuat_longANS_v2.csv", index_col=0)
# df = df[:100]
df_answer

,url,question,long_question,long_answer
0,https://hethongphapluat.com/su-dung-thong-tin-...,Sử dụng thông tin của người khác để vay tiền?,Xin chào ạ! Em xin được hỏi và mong được giải ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
1,https://hethongphapluat.com/cap-lai-chung-minh...,Cấp lại chứng minh thư nhân dân (Căn cước công...,"Thưa luật sư, em tên Thị Nhân là dân tộc Khmer...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...
2,https://hethongphapluat.com/co-duoc-thay-doi-t...,Có được thay đổi từ ngữ khi trích dẫn nội dung...,Luật sư cho em hỏi: Khi mình trích dẫn nội dun...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
3,https://hethongphapluat.com/co-the-nho-nguoi-t...,Có thể nhờ người thân đứng tên trên giấy ủy qu...,NaN,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
4,https://hethongphapluat.com/co-duoc-kien-nguoi...,Có được kiện người đánh mình sau khi đã nhận t...,"Năm 2021, cháu tôi bị người làng đến nhà đánh,...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...
...,...,...,...,...
24000,https://hethongphapluat.com/quy-dinh-ve-ghi-nh...,Quy định về ghi nhãn hàng hóa trên thuốc thú y...,Tôi muốn nhập khẩu một lô thuốc thú y từ Ấn Độ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24001,https://hethongphapluat.com/the-chap-quyen-su-...,Thế chấp quyền sử dụng đất là tài sản của hộ g...,Sổ đỏ nhà tôi được cấp năm 2004 cho hộ gia đìn...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24002,https://hethongphapluat.com/co-duoc-the-chap-t...,Có được thế chấp tài sản thừa kế không? Ngân h...,Bố mẹ tôi lập di chúc chung để lại tài sản là ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24003,https://hethongphapluat.com/giao-dich-vo-hieu-...,Giao dịch vô hiệu khi bị cưỡng ép ký kết hợp đ...,"Bị ép kí vào giấy vay tiền, bị thưa ra tòa và ...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...


In [5]:
# show how many nan of each column
df_answer.isnull().sum()

url               0
question          3
long_question    56
long_answer       1
dtype: int64

In [6]:
df.loc[1]['url']

'https://hethongphapluat.com/cap-lai-chung-minh-thu-nhan-dan-can-cuoc-cong-dan-bi-mat.html'

In [3]:
def crawl_each_link(url):
    driver.get(url)
    wait = WebDriverWait(driver, 1)
    content = driver.find_element(By.XPATH, "//div[@class='panel-body']")

    return content.text



def crawl_parallel(i):
    content = crawl_each_link(df.iloc[i]['url'])
    # print("this is number ", i)
    df.at[i, 'long_answer'] = content
#     print(df.at[i, 'long_answer'])
    return content

def crawl_dask(url):
    content = crawl_each_link(url)
    return content

In [8]:
crawl_each_link(df.loc[1]['url'])

'Hệ thống pháp luật Việt Nam (hethongphapluat.com) xin chân thành cảm ơn bạn đã tin tưởng gửi câu hỏi đến chúng tôi. Sau khi nghiên cứu, đối chiếu với quy định của pháp luật hiện hành, chúng tôi xin đưa ra câu trả lời có tính chất tham khảo như sau:\n1. Căn cứ pháp lý\nLuật căn cước công dân 2014\n2. Nội dung tư vấn\nTrường hợp bạn bị mất CMTND (nay là căn cước công dân- CCCD) nhưng biết rõ người cất giữ thì có quyền yêu cầu trả lại CMTND của bạn đã bị chiếm giữ trái pháp luật.\nTrường hợp bạn bị mất CMTND nhưng không biết rõ ai là người cất giữ CMTND thì căn cứ khoản 1 Điều 22 Luật căn cước công dân 2014 bạn đến công an quận, huyện nơi bạn cư trú thực hiện các bước như sau:\n- Ban điền vào tờ khai theo mẫu quy định;\n- Sau đó người được giao nhiệm vụ chụp ảnh, lăn vân tay thu thập, cập nhật thông tin, tài liệu, kiểm tra, đối chiếu thông tin từ Cơ sở dữ liệu quốc gia về dân cư để xác định chính xác người cần cấp thẻ Căn cước công dân;\n- Cán bộ cơ quan quản lý căn cước công dân cấp giấ

In [7]:
df_answer

,url,question,long_question,long_answer
0,https://hethongphapluat.com/su-dung-thong-tin-...,Sử dụng thông tin của người khác để vay tiền?,Xin chào ạ! Em xin được hỏi và mong được giải ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
1,https://hethongphapluat.com/cap-lai-chung-minh...,Cấp lại chứng minh thư nhân dân (Căn cước công...,"Thưa luật sư, em tên Thị Nhân là dân tộc Khmer...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...
2,https://hethongphapluat.com/co-duoc-thay-doi-t...,Có được thay đổi từ ngữ khi trích dẫn nội dung...,Luật sư cho em hỏi: Khi mình trích dẫn nội dun...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
3,https://hethongphapluat.com/co-the-nho-nguoi-t...,Có thể nhờ người thân đứng tên trên giấy ủy qu...,NaN,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
4,https://hethongphapluat.com/co-duoc-kien-nguoi...,Có được kiện người đánh mình sau khi đã nhận t...,"Năm 2021, cháu tôi bị người làng đến nhà đánh,...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...
...,...,...,...,...
24000,https://hethongphapluat.com/quy-dinh-ve-ghi-nh...,Quy định về ghi nhãn hàng hóa trên thuốc thú y...,Tôi muốn nhập khẩu một lô thuốc thú y từ Ấn Độ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24001,https://hethongphapluat.com/the-chap-quyen-su-...,Thế chấp quyền sử dụng đất là tài sản của hộ g...,Sổ đỏ nhà tôi được cấp năm 2004 cho hộ gia đìn...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24002,https://hethongphapluat.com/co-duoc-the-chap-t...,Có được thế chấp tài sản thừa kế không? Ngân h...,Bố mẹ tôi lập di chúc chung để lại tài sản là ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24003,https://hethongphapluat.com/giao-dich-vo-hieu-...,Giao dịch vô hiệu khi bị cưỡng ép ký kết hợp đ...,"Bị ép kí vào giấy vay tiền, bị thưa ra tòa và ...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...


In [8]:
df_answer.to_csv("./raw_data/raw_HeThongPhapLuat_longANS.csv")

In [6]:
try:
    for i in tqdm(range(22892, len(df))):
        temp_content = crawl_each_link(df.loc[i]['url'])
        df_answer.at[i, 'long_answer'] = temp_content
        # df = pd.concat([df, temp_d])
    driver.close()
    print("Good job! Done and saving to csv")
    df_answer.to_csv("./raw_data/raw_HeThongPhapLuat_longANS.csv")
except Exception as e:
    print(e)
    print("ERROR DUMP at index {}! Saving to csv".format(i));
    df_answer.to_csv("./raw_data/raw_HeThongPhapLuat_longANS.csv")

100%|██████████| 1113/1113 [42:00<00:00,  2.26s/it] 


Good job! Done and saving to csv


In [15]:
from playsound import playsound

# for playing note.wav file
playsound('./oof.mp3')


    Error 259 for command:
        play ./oof.mp3 wait
    The driver cannot recognize the specified command parameter.

    Error 263 for command:
        close ./oof.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: ./oof.mp3


PlaysoundException: 
    Error 259 for command:
        play ./oof.mp3 wait
    The driver cannot recognize the specified command parameter.

In [13]:
df_dask = dd.from_pandas(df[['url']], npartitions=6)
df_dask['long_answer'] = df_dask['url'].apply(crawl_dask, meta=('token2', 'object'))


In [65]:
df_dask.compute()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=108.0.5359.126)
Stacktrace:
Backtrace:
	(No symbol) [0x0065F243]
	(No symbol) [0x005E7FD1]
	(No symbol) [0x004DD04D]
	(No symbol) [0x004CF5CD]
	(No symbol) [0x004E30DD]
	(No symbol) [0x0053CDCE]
	(No symbol) [0x00528386]
	(No symbol) [0x0050163C]
	(No symbol) [0x0050269D]
	GetHandleVerifier [0x008F9A22+2655074]
	GetHandleVerifier [0x008ECA24+2601828]
	GetHandleVerifier [0x00708C0A+619850]
	GetHandleVerifier [0x00707830+614768]
	(No symbol) [0x005F05FC]
	(No symbol) [0x005F5968]
	(No symbol) [0x005F5A55]
	(No symbol) [0x0060051B]
	BaseThreadInitThunk [0x76656BD9+25]
	RtlGetFullPathName_UEx [0x77348FD2+1218]
	RtlGetFullPathName_UEx [0x77348F9D+1165]


In [35]:

df_dask['long_answer'] = df_dask.apply(lambda x: crawl_parallel(x), meta=('str')).compute(scheduler='multiprocessing')
df = pd.DataFrame(df_dask)
df.to_csv('./raw_data/test_parallel')

c:\users\thanh\appdata\local\programs\python\python37\lib\site-packages\dask\dataframe\core.py:6088: FutureWarning: Meta is not valid, `map_partitions` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  FutureWarning,


TypeError: can't pickle _thread.lock objects

# Pre processing

In [3]:
import pandas as pd
import polars as pl

In [4]:
df = pd.read_csv("./raw_data/raw_HeThongPhapLuat_ANS.csv", index_col=0)
df_answer = pd.read_csv("./raw_data/raw_HeThongPhapLuat_longANS.csv", index_col=0)
# df = df[:100]
df_answer

,url,question,long_question,long_answer
0,https://hethongphapluat.com/su-dung-thong-tin-...,Sử dụng thông tin của người khác để vay tiền?,Xin chào ạ! Em xin được hỏi và mong được giải ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
1,https://hethongphapluat.com/cap-lai-chung-minh...,Cấp lại chứng minh thư nhân dân (Căn cước công...,"Thưa luật sư, em tên Thị Nhân là dân tộc Khmer...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...
2,https://hethongphapluat.com/co-duoc-thay-doi-t...,Có được thay đổi từ ngữ khi trích dẫn nội dung...,Luật sư cho em hỏi: Khi mình trích dẫn nội dun...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
3,https://hethongphapluat.com/co-the-nho-nguoi-t...,Có thể nhờ người thân đứng tên trên giấy ủy qu...,NaN,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
4,https://hethongphapluat.com/co-duoc-kien-nguoi...,Có được kiện người đánh mình sau khi đã nhận t...,"Năm 2021, cháu tôi bị người làng đến nhà đánh,...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...
...,...,...,...,...
24000,https://hethongphapluat.com/quy-dinh-ve-ghi-nh...,Quy định về ghi nhãn hàng hóa trên thuốc thú y...,Tôi muốn nhập khẩu một lô thuốc thú y từ Ấn Độ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24001,https://hethongphapluat.com/the-chap-quyen-su-...,Thế chấp quyền sử dụng đất là tài sản của hộ g...,Sổ đỏ nhà tôi được cấp năm 2004 cho hộ gia đìn...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24002,https://hethongphapluat.com/co-duoc-the-chap-t...,Có được thế chấp tài sản thừa kế không? Ngân h...,Bố mẹ tôi lập di chúc chung để lại tài sản là ...,Hệ thống pháp luật Việt Nam (hethongphapluat.c...
24003,https://hethongphapluat.com/giao-dich-vo-hieu-...,Giao dịch vô hiệu khi bị cưỡng ép ký kết hợp đ...,"Bị ép kí vào giấy vay tiền, bị thưa ra tòa và ...",Hệ thống pháp luật Việt Nam (hethongphapluat.c...


In [3]:
# Constant header
CONSTANT_HEADER = ['Hệ thống pháp luật Việt Nam (hethongphapluat.com) xin chân thành cảm ơn bạn đã tin tưởng gửi câu hỏi đến chúng tôi. Sau khi nghiên cứu, đối chiếu với quy định của pháp luật hiện hành, chúng tôi xin đưa ra câu trả lời có tính chất tham khảo như sau:','Trên đây là câu trả lời của Hệ Thống Pháp Luật Việt Nam liên quan đến yêu cầu câu hỏi của bạn. Hy vọng câu trả lời của chúng tôi sẽ hữu ích cho bạn.',
 'Nếu có bất cứ vướng mắc gì về pháp lý mời bạn tiếp tục đặt câu hỏi. Chúng tôi luôn sẵn sàng giải đáp.',
  'Trân trọng cảm ơn.',
 'BBT.Hệ Thống Pháp Luật Việt nam',
 'Lưu ý: Nội dung tư vấn của Hệ Thống Pháp Luật Việt Nam trên đây chỉ mang tính tham khảo. Tùy vào từng thời điểm và đối tượng khác nhau mà nội dung tư vấn trên có thể sẽ không còn phù hợp do sự thay đổi của chính sách hay quy định mới của pháp luật.',
 'Gửi câu hỏi cho luật sư']


In [4]:
# replace all the constant header to empty string
def replace_header(text):
    for header in CONSTANT_HEADER:
        text = text.replace(header, '')
    return text

In [5]:
replace_header(df_answer['long_answer'][1]).strip().split('\n')

['1. Căn cứ pháp lý',
 'Luật căn cước công dân 2014',
 '2. Nội dung tư vấn',
 'Trường hợp bạn bị mất CMTND (nay là căn cước công dân- CCCD) nhưng biết rõ người cất giữ thì có quyền yêu cầu trả lại CMTND của bạn đã bị chiếm giữ trái pháp luật.',
 'Trường hợp bạn bị mất CMTND nhưng không biết rõ ai là người cất giữ CMTND thì căn cứ khoản 1 Điều 22 Luật căn cước công dân 2014 bạn đến công an quận, huyện nơi bạn cư trú thực hiện các bước như sau:',
 '- Ban điền vào tờ khai theo mẫu quy định;',
 '- Sau đó người được giao nhiệm vụ chụp ảnh, lăn vân tay thu thập, cập nhật thông tin, tài liệu, kiểm tra, đối chiếu thông tin từ Cơ sở dữ liệu quốc gia về dân cư để xác định chính xác người cần cấp thẻ Căn cước công dân;',
 '- Cán bộ cơ quan quản lý căn cước công dân cấp giấy hẹn trả thẻ Căn cước công dân cho người đến làm thủ tục;',
 '- Công an quận, huyện sẽ trả thẻ Căn cước công dân theo thời hạn và địa điểm trong giấy hẹn theo quy định; trường hợp bạn có yêu cầu trả thẻ tại địa điểm khác thì cơ

## drop nan

In [7]:
# drop nan by long_answer
df_answer = df_answer.dropna(subset=['long_answer'])

## clean the long_answer

In [8]:
df_pl = pl.DataFrame(df_answer)
df_pl

url,question,long_question,long_answer
str,str,str,str
"""https://hethon…","""Sử dụng thông …","""Xin chào ạ! Em…","""Hệ thống pháp …"
"""https://hethon…","""Cấp lại chứng …","""Thưa luật sư, …","""Hệ thống pháp …"
"""https://hethon…","""Có được thay đ…","""Luật sư cho em…","""Hệ thống pháp …"
"""https://hethon…","""Có thể nhờ ngư…",null,"""Hệ thống pháp …"
"""https://hethon…","""Có được kiện n…","""Năm 2021, cháu…","""Hệ thống pháp …"
"""https://hethon…","""Giải đáp một s…","""Xin chào Luật …","""Hệ thống pháp …"
"""https://hethon…","""Làm CCCD nhưng…","""Mẹ em làm thẻ …","""Hệ thống pháp …"
"""https://hethon…","""Sử dụng phần m…","""Tôi có 2 con đ…","""Hệ thống pháp …"
"""https://hethon…","""Có phải thay đ…","""Chào luật sư, …","""Hệ thống pháp …"


In [14]:
df_pl.apply(lambda x: (replace_header(x[3]).strip()))


apply
str
"""1. Căn cứ pháp…"
"""1. Căn cứ pháp…"
"""1. Căn cứ pháp…"
"""1. Căn cứ pháp…"
"""1. Căn cứ pháp…"
"""Câu hỏi 1: Tôi…"
"""1. Căn cứ pháp…"
"""1. Căn cứ pháp…"
"""1. Căn cứ pha…"


In [17]:
df_answer['long_answer'] = df_answer['long_answer'].apply(lambda x: (replace_header(x).strip()))

D:\tailieu\DOAN\crawlingdata\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:


# df_answer['single_question'] = df_answer['long_answer'].apply(lambda x: x.split('\n')[0])


,url,question,long_question,long_answer
0,https://hethongphapluat.com/su-dung-thong-tin-...,Sử dụng thông tin của người khác để vay tiền?,Xin chào ạ! Em xin được hỏi và mong được giải ...,1. Căn cứ pháp lý\nLuật an toàn thông tin mạng...
1,https://hethongphapluat.com/cap-lai-chung-minh...,Cấp lại chứng minh thư nhân dân (Căn cước công...,"Thưa luật sư, em tên Thị Nhân là dân tộc Khmer...",1. Căn cứ pháp lý\nLuật căn cước công dân 2014...
2,https://hethongphapluat.com/co-duoc-thay-doi-t...,Có được thay đổi từ ngữ khi trích dẫn nội dung...,Luật sư cho em hỏi: Khi mình trích dẫn nội dun...,1. Căn cứ pháp lý\nLuật ban hành văn bản quy p...
3,https://hethongphapluat.com/co-the-nho-nguoi-t...,Có thể nhờ người thân đứng tên trên giấy ủy qu...,NaN,1. Căn cứ pháp lý\nBộ luật dân sự 2015\n2. Nội...
4,https://hethongphapluat.com/co-duoc-kien-nguoi...,Có được kiện người đánh mình sau khi đã nhận t...,"Năm 2021, cháu tôi bị người làng đến nhà đánh,...",1. Căn cứ pháp lý\nBộ luật Hình sự 2015\nBộ lu...
...,...,...,...,...
24000,https://hethongphapluat.com/quy-dinh-ve-ghi-nh...,Quy định về ghi nhãn hàng hóa trên thuốc thú y...,Tôi muốn nhập khẩu một lô thuốc thú y từ Ấn Độ...,Đối với trường hợp của bạn là hàng nhập khẩu t...
24001,https://hethongphapluat.com/the-chap-quyen-su-...,Thế chấp quyền sử dụng đất là tài sản của hộ g...,Sổ đỏ nhà tôi được cấp năm 2004 cho hộ gia đìn...,Điều 179 Luật Đất đai 2013 quy định:\n“Hộ gia ...
24002,https://hethongphapluat.com/co-duoc-the-chap-t...,Có được thế chấp tài sản thừa kế không? Ngân h...,Bố mẹ tôi lập di chúc chung để lại tài sản là ...,"Trong trường hợp của bạn, bố mẹ bạn đã lập di ..."
24003,https://hethongphapluat.com/giao-dich-vo-hieu-...,Giao dịch vô hiệu khi bị cưỡng ép ký kết hợp đ...,"Bị ép kí vào giấy vay tiền, bị thưa ra tòa và ...",Hợp đồng vay tài sản là sự thoả thuận giữa các...
